In [87]:
import os

import cv2
import numpy as np
from dotenv import load_dotenv

# Classification for the base classes

load_dotenv()

# Specify the path to the dataset directory
dataset_dir = os.getenv('DATASET_DIR')

target_size = (128, 128)

In [88]:
# Get all images
def read_images_from_directory(directory):
    images = []
    labels = []

    class_names = []
    for root, dirs, files in os.walk(directory):
        for class_name in dirs:
            class_directory = os.path.join(root, class_name)
            for filename in os.listdir(class_directory):
                file_path = os.path.join(class_directory, filename)
                if os.path.isfile(file_path):
                    image = cv2.imread(file_path, cv2.COLOR_BGR2GRAY)
                    if image is not None:
                        image = cv2.resize(image, target_size)
                        images.append(image)
                        labels.append(os.path.basename(root) + "." + class_name)
                        if class_name not in class_names:
                            class_names.append(class_name)

    images = np.array(images)
    labels = np.array(labels)
    class_names = np.array(class_names)

    return images, labels, class_names

In [89]:
images, labels, class_names = read_images_from_directory(dataset_dir)
images_with_labels = list(zip(images, labels))
print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Class names:", class_names)
print("Labels:", labels)

Number of images: 21
Number of labels: 21
Class names: ['big' 'small' 'agriculture_vehicle' 'bus' 'sportscar' 'curvy' 'adult'
 'child' 'handicaped' 'senior' 'danger' 'information' 'restriction'
 'speed_limit' 'yield' 'green' 'red' 'yellow']
Labels: ['animal.big' 'animal.small' 'car.agriculture_vehicle' 'car.bus'
 'car.sportscar' 'lane.curvy' 'pedestrian.adult' 'pedestrian.child'
 'pedestrian.handicaped' 'pedestrian.handicaped' 'pedestrian.senior'
 'traffic_sign.danger' 'traffic_sign.information'
 'traffic_sign.restriction' 'traffic_sign.restriction'
 'traffic_sign.restriction' 'traffic_sign.speed_limit'
 'traffic_sign.yield' 'trafic_light.green' 'trafic_light.red'
 'trafic_light.yellow']


In [90]:
print("First image: ", images_with_labels[0][0])
print("First image - label: ", images_with_labels[0][1])

First image:  [[[ 18  36  44]
  [ 11  30  34]
  [ 12  31  35]
  ...
  [  7  19  27]
  [  9  21  29]
  [ 15  26  34]]

 [[ 13  30  33]
  [ 12  39  45]
  [ 10  39  44]
  ...
  [ 10  20  27]
  [  9  21  29]
  [ 14  26  32]]

 [[ 11  32  34]
  [ 21  57  60]
  [ 20  56  62]
  ...
  [ 11  22  36]
  [ 11  23  32]
  [ 11  23  33]]

 ...

 [[ 66 149 142]
  [ 75 163 158]
  [ 64 148 146]
  ...
  [ 40  99  83]
  [ 44 106  93]
  [ 33  89  78]]

 [[ 62 147 140]
  [ 59 149 142]
  [ 63 156 144]
  ...
  [ 54 102  96]
  [ 17  82  71]
  [ 66 132 133]]

 [[ 68 152 141]
  [ 60 146 138]
  [ 77 151 154]
  ...
  [ 41 111 103]
  [ 50 118 113]
  [ 45 117 110]]]
First image - label:  animal.big


In [91]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_images, test_images, train_labels0, test_labels0 = train_test_split(images, labels, test_size=0.2,
                                                                          random_state=42)

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))

print(num_classes)

label_encoder = LabelEncoder()
train_labels1 = label_encoder.fit_transform(train_labels0)
test_labels1 = label_encoder.fit_transform(test_labels0)

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels1, num_classes)
test_labels = to_categorical(test_labels1, num_classes)

18


In [93]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional and pooling layers
# Fixme
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more convolutional and pooling layers if desired

# Flatten the output from the previous layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ValueError: Input 0 of layer "conv2d_1" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 128, 128)

In [ ]:
num_epochs = 1
batch_size = 10

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')